# Time series cross validation: Autoregression with seasonal indexes and holidays.

This notebook develops Autoregression models to forecast daily Ambulance Response numbers.  

Autoregressive models have the following features:

* They contain lagged dependent variables
* As the model is a regression it can contain season dummy variables at multiple levels in order to capture multiple seasonalitys

Note within this Notebook we will ignore correlated errors.  The most likely impact of this is that prediction intervals
are too narrow.  A less likely, but possible outcome is that there may also be additional information that improve forecating in the errors.

The final section of the notebook analyses prediction interval coverage.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from pandas.plotting import lag_plot
import seaborn as sns

#forecast error metrics
from forecast_tools.metrics import (mean_absolute_scaled_error, 
                                    root_mean_squared_error,
                                    symmetric_mean_absolute_percentage_error)


#'sklearn'
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, LinearRegression

#statsmodels api
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

In [2]:
sm.__version__

'0.11.0'

In [3]:
from amb_forecast.feature_engineering import featurize_time_series

# Data Input

The constants `TOP_LEVEL`, `STAGE`, `REGION`,`TRUST` and `METHOD` are used to control data selection and the directory for outputting results.  

> Output file is `f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv'.csv`.  where metric will be smape, rmse, mase, coverage_80 and coverage_95. Note: `REGION`: is also used to select the correct data from the input dataframe.

In [4]:
TOP_LEVEL = '../../../results/model_selection'
STAGE = 'stage1'
REGION = 'Trust'
METHOD = 'ar'

FILE_NAME = 'Daily_Responses_5_Years_2019_full.csv'

#split training and test data.
TEST_SPLIT_DATE = '2019-01-01'

#second subdivide: train and val
VAL_SPLIT_DATE = '2017-07-01'

#discard data after 2020 due to coronavirus
#this is the subject of a seperate study.
DISCARD_DATE = '2020-01-01'

In [5]:
#read in path
path = f'../../../data/{FILE_NAME}'

In [6]:
def pre_process_daily_data(path, index_col, by_col, 
                           values, dayfirst=False):
    '''
    Daily data is stored in long format.  Read in 
    and pivot to wide format so that there is a single 
    colmumn for each regions time series.
    '''
    df = pd.read_csv(path, index_col=index_col, parse_dates=True, 
                     dayfirst=dayfirst)
    df.columns = map(str.lower, df.columns)
    df.index.rename(str(df.index.name).lower(), inplace=True)
    
    clean_table = pd.pivot_table(df, values=values.lower(), 
                                 index=[index_col.lower()],
                                 columns=[by_col.lower()], aggfunc=np.sum)
    
    clean_table.index.freq = 'D'
    
    return clean_table

In [7]:
clean = pre_process_daily_data(path, 'Actual_dt', 'ORA', 'Actual_Value', 
                               dayfirst=False)
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Train Test Split

In [8]:
def ts_train_test_split(data, split_date):
    '''
    Split time series into training and test data
    
    Parameters:
    -------
    data - pd.DataFrame - time series data.  Index expected as datatimeindex
    split_date - the date on which to split the time series
    
    Returns:
    --------
    tuple (len=2) 
    0. pandas.DataFrame - training dataset
    1. pandas.DataFrame - test dataset
    '''
    train = data.loc[data.index < split_date]
    test = data.loc[data.index >= split_date]
    return train, test

In [9]:
train, test = ts_train_test_split(clean, split_date=TEST_SPLIT_DATE)

#exclude data after 2020 due to coronavirus.
test, discard = ts_train_test_split(test, split_date=DISCARD_DATE)

#split into train and val AFTER creating new years day.

In [10]:
train.shape

(1828, 9)

In [11]:
test.shape

(365, 9)

# autoregressive lags, seasonal indexes and New years day

Generate lags + new binary categorical feature representing new years day.

In [33]:
#exclude interaction as point forecasts are less accurate.
lagged, calendar_dummies, new_year = featurize_time_series(train[REGION], 
                                                    max_lags=7, 
                                                    include_interactions=False)

In [34]:
#rename column and drop quarters and t from seasonal indexes
new_year.columns = ['new_year']
calendar_dummies = calendar_dummies[calendar_dummies.columns[:-4]]

In [35]:
#combined to single dataframe
processed = pd.concat([train[REGION], lagged, calendar_dummies, new_year], 
                      axis=1)
processed.head()

,actual,actual_lag1,actual_lag2,actual_lag3,actual_lag4,actual_lag5,actual_lag6,actual_lag7,m_2,m_3,...,m_10,m_11,m_12,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,new_year
actual_dt,,,,,,,,,,,,,,,,,,,,,
2013-12-30,2042.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,1,0,0,0,0,0,0,0
2013-12-31,1996.0,2042.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,1,1,0,0,0,0,0,0
2014-01-01,2570.0,1996.0,2042.0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,1,0,0,0,0,1
2014-01-02,2013.0,2570.0,1996.0,2042.0,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,1,0,0,0,0
2014-01-03,2056.0,2013.0,2570.0,1996.0,2042.0,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,1,0,0,0


# Train validation split

In [39]:
#train split into train and validation
train, val = ts_train_test_split(processed, split_date=VAL_SPLIT_DATE)

In [40]:
train.shape

(1279, 26)

In [41]:
val.shape

(549, 26)

## Cross Validation

`time_series_cv` implements rolling forecast origin cross validation for time series.  
It does not calculate forecast error, but instead returns the predictions, pred intervals and actuals in an array that can be passed to any forecast error function. (this is for efficiency and allows additional metrics to be calculated if needed).


> Note: prediction uses an iterative method where ground truth inputs are gradually replaced with forecast values.

In [43]:
def time_series_cv(train, val, horizons, lags, step=1, alpha=0.2, 
                   regularised=False, reg_weight=0.1):
    '''
    Time series cross validation across multiple horizons for a autoregressive
    model.

    Incrementally adds additional training data to the model and tests
    across a provided list of forecast horizons. Note that function tests a
    model only against complete validation sets.  E.g. if horizon = 15 and 
    len(val) = 12 then no testing is done.  In the case of multiple horizons
    e.g. [7, 14, 28] then the function will use the maximum forecast horizon
    to calculate the number of iterations i.e if len(val) = 365 and step = 1
    then no. iterations = len(val) - max(horizon) = 365 - 28 = 337.
    
    Note that when the forecast horizon exceeds the lag number of the model
    forecasts will be based of previous forecasts.  I.e. for a lag 7 model the first 7 
    forecasts will include at least one ground truth observation, but from step 8 onwards all 
    forecasts iwll be based on the previous 7 predicted y values.
    
    Parameters:
    --------
    train - np.array - vector of training data

    val - np.array - vector of validation data

    horizon - list of ints, forecast horizon e.g. [7, 14, 28] days

    step -- step taken in cross validation 
            e.g. 1 in next cross validation training data includes next point 
            from the validation set.
            e.g. 7 in the next cross validation training data includes next 7 points
            (default=1)
            
    Returns:
    -------
    np.array - vector of forecast errors from the CVs.
    '''
    cv_preds = []
    cv_actuals = []
    cv_intervals = []
    MAX_LAG = lags
    split = 1
    
    print('split => ', end="")
    for i in range(0, len(val) - max(horizons) + 1, step):
        print(f'{split}, ', end="")
        split += 1
        
        #create and fit on a new training set (train + val)
        train_cv = np.concatenate([train, val.iloc[:i, 0:]], axis=0) 
        X_train, y_train = train_cv[:,1:], train_cv[:,0]       
        
        # Fit and summarize OLS model
        X_train = sm.add_constant(X_train, prepend=False, has_constant='add')
        model = sm.OLS(endog=y_train, exog=X_train)
        
        if not regularised:
            results = model.fit()
        else:    
            results = model.fit_regularized(method='elastic_net',
                                            alpha=reg_weight, refit=True)

        preds = []
        intervals = []
        current_batch = val.iloc[i, 1:].to_numpy()

        #iteratively predict the next data point
        #and update the lags.  Remember that the
        #actual lags are replaced by FORECAST values 
        #as time goes on.
        for j in range(max(horizons)): 
            
            #one timestep ahead of historical points
            batch_exog = current_batch

            batch_exog = sm.add_constant(batch_exog.reshape(-1,1).T, 
                                         prepend=False,
                                         has_constant='add')
                       
            predictions = results.get_prediction(batch_exog)
            
            df = predictions.summary_frame(alpha=alpha)
            
            y_pred = df['mean'].to_numpy()
            preds.append(y_pred)
            
            #prediction intervals
            y_intervals = df[['obs_ci_lower', 'obs_ci_upper']].to_numpy()
            intervals.append(y_intervals)
            
            #remove tail lagged value add prediction to head 
            current_batch[:MAX_LAG] = np.append(y_pred, current_batch[:MAX_LAG-1])
            
            #get next set of seasonal dummies
            current_batch[MAX_LAG:] = val.iloc[i+j+1, MAX_LAG+1:].to_numpy()
        
        cv_h_preds = []
        cv_h_intervals = []
        cv_test = []
                
        for h in horizons:
            #store the h-step mean prediction
            cv_h_preds.append(np.concatenate(preds[:h]))
            
            #store the h-step prediction intervals
            cv_h_intervals.append(np.concatenate(intervals[:h]))
            #store the h-step actual value
            cv_test.append(val.iloc[i:i+h,0])                 
                     
        cv_preds.append(cv_h_preds)
        cv_intervals.append(cv_h_intervals)
        cv_actuals.append(cv_test)
        
    
    print('done.\n')
    return cv_preds, cv_intervals, cv_actuals

## Cross validation (without regularization)

Time horizons evaluated (in days) are 7, 14, 28, 56, 84, 365.

Note: These are basic regression models and do not have an ARIMA error process.  This means that interpretation of the coefficients is problematic and should be done with caution.

**All models contain the seasonal dummy variables, t and exceptional days.**

Models evaluated:

* lags 1
* lags 1, 2
* lags 1, 2, 3
* lags 1, 2, 3, .... `MAX_LAGS`

Notes: the cross validation function is returning the error measure over the different time horizons.  So the summary statistics refer to the distrubution of the test statistic.  Not the individual forecast errors of each point.

In [44]:
def split_cv_error(cv_preds, cv_test, error_func):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = error_func(cv_test[split], cv_preds[split])
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv(cv_preds, cv_test, error_func):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error(cv_preds[h], cv_test[h], error_func)
        horizon_errors.append(split_errors)

    return np.array(horizon_errors)

def split_coverage(cv_test, cv_intervals):
    n_splits = len(cv_test)
    cv_errors = []
        
    for split in range(n_splits):
        val = np.asarray(cv_test[split])
        lower = cv_intervals[split].T[0]
        upper = cv_intervals[split].T[1]
        
        coverage = len(np.where((val > lower) & (val < upper))[0])
        coverage = coverage / len(val)
        
        cv_errors.append(coverage)
        
    return np.array(cv_errors)
    
    
def prediction_int_coverage_cv(cv_test, cv_intervals):
    cv_test = np.array(cv_test)
    cv_intervals = np.array(cv_intervals)
    n_horizons = len(cv_test)    
    
    horizon_coverage = []
    for h in range(n_horizons):
        split_coverages = split_coverage(cv_test[h], cv_intervals[h])
        horizon_coverage.append(split_coverages)

    return np.array(horizon_coverage)  

In [45]:
def split_cv_error_scaled(cv_preds, cv_test, y_train):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = mean_absolute_scaled_error(cv_test[split], cv_preds[split], 
                                                y_train, period=7)
        
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv_scaled(cv_preds, cv_test, y_train):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error_scaled(cv_preds[h], cv_test[h], y_train)
        horizon_errors.append(split_errors)
        
    return np.array(horizon_errors)

In [47]:


MAX_LAG = 7
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
STEP = 7
PREFIX = 'actual'

y_lags = [PREFIX+'_lag' + str(i) for i in range(1, MAX_LAG+1)]

select_cv = ['actual'] + y_lags + list(calendar_dummies.columns) + \
    list(new_year.columns)


cv_preds, cv_intervals, cv_test  = time_series_cv(train=train[select_cv][MAX_LAG+1:], 
                                                  val=val[select_cv], 
                                                  horizons=horizons,
                                                  lags=len(y_lags),
                                                  step=STEP,
                                                  alpha=0.2)



split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [50]:
cv_intervals[0][0]

array([[2191.82227067, 2370.77318012],
       [2163.29042123, 2342.20535038],
       [2066.33364772, 2245.26333601],
       [1986.41480171, 2165.22800952],
       [1978.7094183 , 2157.47054199],
       [2004.52190743, 2183.27170169],
       [2056.35436403, 2234.91403217]])

In [51]:
#CV point predictions smape
cv_errors = forecast_errors_cv(cv_preds, cv_test, 
                               symmetric_mean_absolute_percentage_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,3.208020,3.563141,3.795126,4.000028,4.159445,4.302911,4.432084,4.565148,4.672183,4.778804,4.891725,4.993684,5.002033
std,1.122335,1.047744,0.882623,0.863666,0.958771,1.029291,1.074982,1.114456,1.124972,1.146078,1.172122,1.168497,0.220911
min,1.503479,1.897222,2.342834,2.562351,2.783483,2.762655,2.702651,2.766886,2.890617,3.041443,3.083533,3.363906,4.490962
25%,2.533541,2.908424,3.281733,3.340787,3.531046,3.477897,3.725477,3.902301,4.036282,4.128845,4.154850,4.135156,4.858556
50%,3.154700,3.340907,3.618547,3.968077,4.154980,4.173193,4.238100,4.380771,4.355394,4.430484,4.611300,4.720808,5.036206
75%,3.639578,4.001279,4.135099,4.662474,4.872152,4.976073,5.237836,5.274810,5.338967,5.606060,5.865711,5.983976,5.190375
max,7.032771,6.416015,5.495095,5.585666,6.412107,6.546856,6.689826,6.794582,7.122329,6.856729,7.134288,7.327150,5.263574


In [52]:
#output sMAPE results to file
metric = 'smape'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-ar_smape.csv


In [53]:
#CV point predictions rmse - no interactions
cv_errors = forecast_errors_cv(cv_preds, cv_test, root_mean_squared_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,83.474784,94.278029,101.658720,107.526136,111.630448,115.044179,118.084666,121.280057,123.884020,126.499772,129.132697,131.502520,133.640602
std,28.062919,28.841003,27.027198,27.052136,28.112013,28.806439,29.164159,29.370553,29.011602,29.101388,29.254349,28.818805,4.155083
min,41.345704,59.869374,57.886822,64.845476,71.293663,75.500070,75.633085,76.522526,78.498440,81.258701,82.865115,90.363652,122.343230
25%,66.801326,79.788352,87.209081,86.780098,91.542663,96.739832,99.857556,105.049755,106.823819,109.482219,110.109865,110.062406,131.281270
50%,76.253369,85.522598,94.440390,102.071490,105.258246,112.319927,111.620164,114.039983,114.726025,114.468002,118.954746,120.626988,134.262632
75%,95.392282,103.468173,111.689863,120.167465,122.107568,131.524221,144.626516,148.373492,147.730523,153.035711,157.033483,157.595905,137.149653
max,187.517781,183.725216,169.363947,164.476784,173.408807,171.580125,171.261639,171.915777,178.884418,175.410655,180.645532,183.892585,138.424338


In [54]:
#output rmse
metric = 'rmse'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-ar_rmse.csv


In [56]:
#mase
cv_errors = forecast_errors_cv_scaled(cv_preds, cv_test, train['actual'])
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.860845,0.957338,1.019508,1.074142,1.116464,1.154558,1.189075,1.224854,1.253678,1.282564,1.313194,1.340829,1.332612
std,0.324613,0.311463,0.270448,0.266984,0.289911,0.305962,0.315856,0.324368,0.325762,0.329634,0.334814,0.332434,0.059583
min,0.395840,0.479859,0.587899,0.646535,0.704888,0.707152,0.690331,0.706663,0.739696,0.781307,0.795670,0.869881,1.191133
25%,0.677464,0.782805,0.859710,0.880520,0.928069,0.910309,0.988603,1.033898,1.062882,1.086559,1.090179,1.084622,1.294507
50%,0.837642,0.902937,0.969446,1.060100,1.102203,1.100208,1.133636,1.161940,1.166508,1.185542,1.217986,1.249135,1.342875
75%,0.977719,1.051680,1.117620,1.243641,1.283127,1.360921,1.472157,1.481519,1.492901,1.552234,1.611535,1.635176,1.382550
max,2.048567,1.912177,1.622885,1.596242,1.773316,1.788436,1.815270,1.837808,1.925149,1.849013,1.926235,1.977432,1.403148


In [57]:
metric = 'mase'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-ar_mase.csv


In [58]:
#80% PIs
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.671958,0.611111,0.573192,0.543651,0.522751,0.502646,0.485261,0.467593,0.452675,0.439153,0.425685,0.413139,0.430137
std,0.208963,0.180846,0.152588,0.140200,0.145646,0.151499,0.148286,0.144227,0.140902,0.139048,0.137079,0.134963,0.040028
min,0.142857,0.214286,0.333333,0.321429,0.285714,0.261905,0.224490,0.214286,0.190476,0.214286,0.194805,0.178571,0.386301
25%,0.571429,0.500000,0.428571,0.446429,0.414286,0.369048,0.377551,0.357143,0.333333,0.342857,0.324675,0.303571,0.394521
50%,0.714286,0.642857,0.619048,0.500000,0.514286,0.500000,0.469388,0.446429,0.428571,0.414286,0.389610,0.404762,0.427397
75%,0.857143,0.750000,0.666667,0.660714,0.600000,0.607143,0.581633,0.562500,0.531746,0.528571,0.519481,0.529762,0.456164
max,1.000000,0.928571,0.857143,0.821429,0.771429,0.761905,0.755102,0.732143,0.714286,0.685714,0.662338,0.630952,0.517808


In [59]:
#write 80% coverage to file
metric = 'coverage_80'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-ar_coverage_80.csv


# Repeat for 95% PIs

In [60]:
#95% PIs
MAX_LAG = 7
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
STEP = 7
PREFIX = 'actual'

y_lags = [PREFIX+'_lag' + str(i) for i in range(1, MAX_LAG+1)]

select_cv = ['actual'] + y_lags + list(calendar_dummies.columns) + \
    list(new_year.columns)


cv_preds, cv_intervals, cv_test  = time_series_cv(train=train[select_cv][MAX_LAG+1:], 
                                                  val=val[select_cv], 
                                                  horizons=horizons,
                                                  lags=len(y_lags),
                                                  step=STEP,
                                                  alpha=0.05)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [61]:
#95% PIs
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.899471,0.857143,0.827160,0.801587,0.775661,0.753968,0.733938,0.714286,0.699001,0.683598,0.668110,0.655203,0.682496
std,0.147266,0.157243,0.148924,0.139146,0.143012,0.149859,0.156193,0.161476,0.158454,0.156778,0.158704,0.155166,0.027798
min,0.428571,0.428571,0.523810,0.571429,0.457143,0.404762,0.387755,0.357143,0.317460,0.357143,0.337662,0.309524,0.646575
25%,0.857143,0.785714,0.714286,0.696429,0.671429,0.690476,0.683673,0.669643,0.650794,0.642857,0.590909,0.559524,0.654795
50%,1.000000,0.928571,0.857143,0.821429,0.800000,0.761905,0.734694,0.714286,0.698413,0.714286,0.701299,0.690476,0.682192
75%,1.000000,0.964286,0.952381,0.928571,0.871429,0.833333,0.836735,0.794643,0.777778,0.764286,0.772727,0.761905,0.704110
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.976190,0.979592,0.964286,0.952381,0.942857,0.948052,0.892857,0.739726


In [62]:
#write 95% coverage to file
metric = 'coverage_95'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-ar_coverage_95.csv


# End